# Generative AI - Agents


In [14]:
!uv pip install -q \
    litellm==1.78.5 \
    python-dotenv==1.1.1 \
    pydantic==2.12.3

In [ ]:
import json
from typing import Any, Callable, Dict, List

import litellm  # type: ignore
from dotenv import load_dotenv  # type: ignore
from pydantic import BaseModel  # type: ignore

load_dotenv()

True

In [ ]:
class Tool(BaseModel):
    name: str
    description: str
    parameters: Dict[str, Any]
    strict: bool

In [ ]:
class AIAgent:
    def __init__(self, max_iterations: int = 10):
        self.messages: List[Dict[str, Any]] = []
        self.tools: List[Tool] = []
        self.max_iterations: int = max_iterations
        self.tool_map: Dict[str, Callable] = {}

    def add_tool_definition(self, tool: Tool):
        self.tools.append(tool)

    def add_tool_function(self, name: str, func: Callable):
        self.tool_map[name] = func

    def execute_tool_function(self, tool_name: str, *args, **kwargs):
        return self.tool_map[tool_name](*args, **kwargs)

    def chat(self, user_input: str):
        self.messages.append({"role": "user", "content": user_input})

        tool_schemas = [
            {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.parameters,
            }
            for tool in self.tools
        ]

        completion = litellm.completion(
            model="gemini/gemini-2.0-flash",
            messages=self.messages,
            tools=tool_schemas,
        )

        self.current_iteration = 0

        while self.current_iteration < self.max_iterations:
            self.current_iteration += 1

            try:
                completion = litellm.completion(
                    model="gemini/gemini-2.0-flash",
                    messages=self.messages,
                    tools=self.tools,
                )

                if not completion.choices:
                    raise Exception("Model returned an empty response.")

                choice = completion.choices[0].message
                tool_calls = getattr(choice, "tool_calls", None)

                if not tool_calls:
                    final_content = choice.content

                    if not final_content:
                        raise Exception(
                            "Agent did not return a response content."
                        )

                    self.messages.append(
                        {"role": "assistant", "content": final_content}
                    )

                    return final_content

                for tool_call in tool_calls:
                    name = tool_call.function.name
                    args = json.loads(tool_call.function.arguments)
                    self.messages.append(completion.choices[0].message)

                    result = self.execute_tool_function(name, **args)
                    self.messages.append(
                        {
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": result,
                        }
                    )

            except Exception as error:
                print(f"Error: {error}")

In [ ]:
greeting_tool = Tool(
    name="greeting",
    description="greet a user",
    parameters={
        "type": "object",
        "properties": {"name": {"type": "string"}},
        "required": ["name"],
        "additionalProperties": False,
    },
    strict=True,
)


def greeting(name: str):
    return f"Hello, {name}"

In [ ]:
agent = AIAgent()
agent.add_tool_definition(greeting_tool)
agent.add_tool_function("greeting", greeting)
agent.execute_tool_function("greeting", "John")

'Hello, John'

In [20]:
while True:
    user_input = input("🧑 You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("👋 Goodbye!")
        break

    response = agent.chat(user_input)
    print(f"🧑 User: {user_input}")
    print(f"🤖 Agent: {response}")

🧑 User: hello
🤖 Agent: I need a name to greet.

🧑 User: john
🤖 Agent: Hello, john

👋 Goodbye!
